# VICReg out-of-distribution class classification benchmark

##Acknowledgments

https://github.com/baxterai/vicreg/tree/branch-distributedExecutionOption (non-distributedExecution of Meta VICReg repository)

* https://github.com/facebookresearch/vicreg

##Setup

###Set up VICreg code, datasets, folder structure;

In [ ]:
#import vicreg code and datasets:
#warning: executing this code block will overwrite all source/data files in memory

from google.colab import drive
drive.mount('/content/gdrive')

!cp -av gdrive/My\ Drive/code/vicreg .
!unzip gdrive/My\ Drive/code/datasets/imagenette2-320.zip
!unzip gdrive/My\ Drive/code/datasets/imagenette2-320-half.zip

#note imagenette2-320-half contains 50% of classes of imagenette2-320 train/val data

In [2]:
#create experiment output folders:
#!mkdir exp
!mkdir expbp
!mkdir expvr

In [ ]:
#import existing/saved models (if available)
#!cp -av gdrive/My\ Drive/code/expbp/* expbp/
#!cp -av gdrive/My\ Drive/code/expvr/* expvr/

###Trial vicreg code (test only based on README.md);

In [ ]:
#Single-node local training
#!python vicreg/main_vicreg-nonDistributed.py --data-dir ./imagenette2-320 --exp-dir ./exp --arch resnet50 --epochs 1 --batch-size 64 --base-lr 0.3

In [ ]:
#Linear evaluation
#!python vicreg/evaluate-nonDistributed.py --data-dir ./imagenette2-320 --pretrained ./exp/resnet50.pth --exp-dir ./exp --epochs 1 --lr-head 0.02

##Test out-of-distribution class classification after Backprop vs VICReg model backbone training;
- expbp: Resnet-50 top 1 performance trained on 50% of images with Backprop, evaluate top-1 performance when only training final layer [ie linear classifier] on 100% images
- expvr: Resnet-50 top 1 performance trained on 50% of images with VICREG, evaluate top-1 performance when only training final layer [ie linear classifier] on 100% images


###expbp (backprop 50% training, 100% linear eval);

In [ ]:
#do not train, just generate model for evaluate input (expbp/resnet50.pth)
!python vicreg/main_vicreg-nonDistributed.py --data-dir ./imagenette2-320-half --exp-dir ./expbp --arch resnet50 --epochs 1 --batch-size 64 --base-lr 0.3

Namespace(arch='resnet50', base_lr=0.3, batch_size=64, cov_coeff=1.0, data_dir=PosixPath('imagenette2-320-half'), device='cuda', epochs=1, exp_dir=PosixPath('expbp'), log_freq_time=60, mlp='8192-8192-8192', num_workers=0, rank=0, sim_coeff=25.0, std_coeff=25.0, wd=1e-06)
vicreg/main_vicreg-nonDistributed.py --data-dir ./imagenette2-320-half --exp-dir ./expbp --arch resnet50 --epochs 1 --batch-size 64 --base-lr 0.3
{"epoch": 0, "step": 12, "loss": 34.452816009521484, "time": 60, "lr": 0.0012}
{"epoch": 0, "step": 27, "loss": 33.82253646850586, "time": 124, "lr": 0.0026999999999999997}
{"epoch": 0, "step": 42, "loss": 30.64160919189453, "time": 187, "lr": 0.0042}
{"epoch": 0, "step": 57, "loss": 27.82965850830078, "time": 250, "lr": 0.005699999999999999}
{"epoch": 0, "step": 72, "loss": 26.33600425720215, "time": 313, "lr": 0.007199999999999999}


In [ ]:
#Backprop hidden layer training with 50% classes
!python vicreg/evaluate-nonDistributed.py --data-dir ./imagenette2-320-half --pretrained ./expbp/resnet50.pth --exp-dir ./expbp --epochs 100 --batch-size 64 --weights finetune --train-perc 100 --lr-backbone 0.3 --lr-head 0.08 --weight-decay 0

Namespace(arch='resnet50', batch_size=64, data_dir=PosixPath('imagenette2-320-half'), epochs=100, exp_dir=PosixPath('expbp'), lr_backbone=0.3, lr_head=0.08, pretrained=PosixPath('expbp/resnet50.pth'), print_freq=100, rank=0, train_percent=100, weight_decay=0.0, weights='finetune', workers=0)
vicreg/evaluate-nonDistributed.py --data-dir ./imagenette2-320-half --pretrained ./expbp/resnet50.pth --exp-dir ./expbp --epochs 100 --batch-size 64 --weights finetune --train-perc 100 --lr-backbone 0.3 --lr-head 0.08 --weight-decay 0
{"epoch": 1, "step": 100, "lr_backbone": 0.2999259840548597, "lr_head": 0.07998026241462927, "loss": 1.3653392791748047, "time": 40}
{"epoch": 1, "acc1": 57.8101456631134, "acc5": 100.0, "best_acc1": 57.8101456631134, "best_acc5": 100.0}
{"epoch": 2, "step": 200, "lr_backbone": 0.29970400926424073, "lr_head": 0.07992106913713087, "loss": 1.0152873992919922, "time": 198}
{"epoch": 2, "acc1": 63.53591160029397, "acc5": 100.0, "best_acc1": 63.53591160029397, "best_acc5":

In [ ]:
#Linear evaluation with 100% classes
!python vicreg/evaluate-nonDistributed.py --data-dir ./imagenette2-320 --pretrained ./expbp/resnet50eval-atEpoch75.pth --exp-dir ./expbp --epochs 100 --batch-size 64 --weights freeze --lr-head 0.02

Namespace(arch='resnet50', batch_size=64, data_dir=PosixPath('imagenette2-320'), epochs=100, exp_dir=PosixPath('expbp'), lr_backbone=0.0, lr_head=0.02, pretrained=PosixPath('expbp/resnet50eval-atEpoch75.pth'), print_freq=100, rank=0, train_percent=100, weight_decay=1e-06, weights='freeze', workers=0)
vicreg/evaluate-nonDistributed.py --data-dir ./imagenette2-320 --pretrained ./expbp/resnet50eval-atEpoch75.pth --exp-dir ./expbp --epochs 100 --batch-size 64 --weights freeze --lr-head 0.02
{"epoch": 0, "step": 0, "lr_backbone": 0, "lr_head": 0.02, "loss": 6.90425968170166, "time": 3}
{"epoch": 0, "step": 100, "lr_backbone": 0, "lr_head": 0.02, "loss": 1.1667380332946777, "time": 42}
{"epoch": 0, "acc1": 67.36305733650353, "acc5": 95.6687898050296, "best_acc1": 67.36305733650353, "best_acc5": 95.6687898050296}
{"epoch": 1, "step": 200, "lr_backbone": 0, "lr_head": 0.019995065603657318, "loss": 0.7716917395591736, "time": 132}
{"epoch": 1, "acc1": 69.80891718384567, "acc5": 96.4076433179332

###expvr (vicreg 50% training, 100% linear eval);

In [ ]:
#VICReg hidden layer training with 50% classes
!python vicreg/main_vicreg-nonDistributed.py --data-dir ./imagenette2-320-half --exp-dir ./expvr --epochs 100 --batch-size 64 --arch resnet50 --batch-size 64 --base-lr 0.3

Namespace(arch='resnet50', base_lr=0.3, batch_size=64, cov_coeff=1.0, data_dir=PosixPath('imagenette2-320-half'), device='cuda', epochs=100, exp_dir=PosixPath('expvr'), log_freq_time=60, mlp='8192-8192-8192', num_workers=0, rank=0, sim_coeff=25.0, std_coeff=25.0, wd=1e-06)
vicreg/main_vicreg-nonDistributed.py --data-dir ./imagenette2-320-half --exp-dir ./expvr --epochs 100 --batch-size 64 --arch resnet50 --batch-size 64 --base-lr 0.3
{"epoch": 0, "step": 14, "loss": 37.9083366394043, "time": 63, "lr": 0.0014}
{"epoch": 0, "step": 30, "loss": 34.594940185546875, "time": 126, "lr": 0.003}
{"epoch": 0, "step": 46, "loss": 29.08403205871582, "time": 188, "lr": 0.0046}
{"epoch": 0, "step": 62, "loss": 27.103389739990234, "time": 251, "lr": 0.006199999999999999}
{"epoch": 1, "step": 76, "loss": 25.7060604095459, "time": 313, "lr": 0.0076}
{"epoch": 1, "step": 92, "loss": 24.989355087280273, "time": 375, "lr": 0.0092}
{"epoch": 1, "step": 108, "loss": 24.293685913085938, "time": 437, "lr": 0.

In [4]:
#Linear evaluation with 100% classes
!python vicreg/evaluate-nonDistributed.py --data-dir ./imagenette2-320 --pretrained ./expvr/resnet50.pth --exp-dir ./expvr --epochs 100 --batch-size 64 --weights freeze --lr-head 0.02

Namespace(arch='resnet50', batch_size=64, data_dir=PosixPath('imagenette2-320'), epochs=100, exp_dir=PosixPath('expvr'), lr_backbone=0.0, lr_head=0.02, pretrained=PosixPath('expvr/resnet50.pth'), print_freq=100, rank=0, train_percent=100, weight_decay=1e-06, weights='freeze', workers=0)
vicreg/evaluate-nonDistributed.py --data-dir ./imagenette2-320 --pretrained ./expvr/resnet50.pth --exp-dir ./expvr --epochs 100 --batch-size 64 --weights freeze --lr-head 0.02
{"epoch": 0, "step": 0, "lr_backbone": 0, "lr_head": 0.02, "loss": 6.897048473358154, "time": 3}
{"epoch": 0, "step": 100, "lr_backbone": 0, "lr_head": 0.02, "loss": 1.964916706085205, "time": 40}
{"epoch": 0, "acc1": 58.165605087766224, "acc5": 93.27388534643089, "best_acc1": 58.165605087766224, "best_acc5": 93.27388534643089}
{"epoch": 1, "step": 200, "lr_backbone": 0, "lr_head": 0.019995065603657318, "loss": 1.9209238290786743, "time": 128}
{"epoch": 1, "acc1": 58.87898088977595, "acc5": 93.80891718384567, "best_acc1": 58.87898